# Google Drive Access

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
!pip install -q pydrive

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Installing Necessary Libs

In [ ]:
!pip install -q keras

In [ ]:
!pip install simpletransformers

# Specifying Project Path

In [ ]:
project_path = '/content/drive/My Drive/Code Documentation Project/'## we will store our data in this drive
project_path_to_save_result = '/content/drive/My Drive/Code Documentation Project/Documentation Smell (Extension)/Code/SANER Model Training and Prediction/'## we will store our data in this drive

# Defining Some Necessary Functions


In [ ]:
import re
import string
import numpy as np
import pandas as pd


def clean_text(text):
    
    text = re.sub(r"\n", " ", text)
    text = re.sub(r"\r", " ", text)


    return text

In [ ]:
from sklearn.metrics import confusion_matrix
def print_confusion_matrix(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    print('True positive = ', cm[0][0])
    print('False positive = ', cm[0][1])
    print('False negative = ', cm[1][0])
    print('True negative = ', cm[1][1])

# Importing Necessary Libs

In [ ]:
from sklearn.model_selection import train_test_split
from simpletransformers.classification import MultiLabelClassificationModel

import glob

import os
#from sklearn.model_selection import KFold
#from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import gc
import keras.backend as K
import numpy as np
import pickle
import pandas as pd
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score

# Loading Dataset

In [ ]:
all_files = glob.glob(project_path+"Documentation Smell (Extension)/Dataset/all labelled sample sets-extension/" + "/*.xlsx")

li = []

main_df = pd.DataFrame()

for filename in all_files:
    df = pd.read_excel(filename)
    try:
      df['Documentation Text'] = df['Documentation Text'].apply(clean_text)
    except:
      print(filename)
      pass
    main_df = main_df.append(df,ignore_index=True)


main_df = main_df.dropna(axis=1, how='any') #deleting (auto-generated) extra columns (containing all NaN values

print(len(main_df))

print(main_df)

# Loading Model

In [ ]:

loaded_model = MultiLabelClassificationModel(
    "bert", project_path + "Labelled Dataset/Models/Transformers/output/bert/"
)

avg_for_result_calculation = 'weighted'

pred, raw_outputs = loaded_model.predict(list(main_df['Documentation Text']))
pred_binary=np.array(pred)



categories_list=['Fragmented','Tangled','Excessive Structured','Bloated','Lazy']

pred_label_df = pd.DataFrame(pred_binary, columns=categories_list)

print(len(pred_label_df))


full_predicted_df = pd.concat([main_df, pred_label_df], axis=1)



print(len(full_predicted_df))

# Result Generation

In [ ]:
for category in categories_list:
  this_category_test_list = main_df[category].tolist()
  this_category_pred_list = pred_label_df[category].tolist()
  acc_for_this_category = accuracy_score(this_category_test_list,this_category_pred_list)
  precision_for_this_category = precision_score(this_category_test_list,this_category_pred_list,average=avg_for_result_calculation)
  recall_for_this_category = recall_score(this_category_test_list,this_category_pred_list,average=avg_for_result_calculation)
  f1_for_this_category = f1_score(this_category_test_list,this_category_pred_list,average=avg_for_result_calculation)
  print('result for '+category+' smell type:')
  print('accuracy: '+ str(acc_for_this_category))
  print('precision: '+ str(precision_for_this_category))
  print('recall: '+ str(recall_for_this_category))
  print('f1 score: '+ str(f1_for_this_category))
  print_confusion_matrix(this_category_test_list,this_category_pred_list)
  print('-------------------------------------------------------------')
  print('')
